詞意有可能過於fine-grained...
已知可剔除無連結者，用https://zh.wikipedia.org/wiki/{title}試試看
有連結者爬取頁面的內容作為訓練檔案

issues: 
1. senses overlap
**建築**底下那塊根本是同個意思...
照理來說應該把section也爬進去，
但這個麻煩的是每個消歧義頁的架構不同，有些會這樣sect開來有些直接混在一起、
有些的架構是開目次有些是新增行列

```
長城 (消歧義)
維基百科，自由的百科全書
長城指古代中國為抵禦不同時期塞北遊牧部落聯盟的侵襲，修築規模浩大的隔離牆或軍事工程的統稱。

長城還可以指：

長城 (天文學)，一個關於宇宙大尺度結構的天文學名詞。
長城 (Beyond)，一首由香港Beyond樂隊創作的歌曲。
長城 (電影)，由張藝謀執導的2016年電影。
**地點**
長城區，中國甘肅省嘉峪關市下轄的一個縣級管理區。
長城郡，大韓民國全羅南道一郡。
**建築**
長城 (雅典)，環繞保護雅典及其海港比雷埃夫斯的防禦工事。
明長城，位於中國北方，明朝修建的長城，西起嘉峪關，東至虎山長城。
齊長城，位於中國山東省境內，春秋戰國時期齊國修建的長城。
魏長城，位於中國陝西省境內，戰國時期魏國修建的長城。
苗疆長城，位於中國湖南省湘西土家族苗族自治州境內，明朝修建的，以防止苗民暴動的長城。
千里長城，位於中國遼東和朝鮮半島北部，由高句麗和高麗王朝修建的長城。
金界壕，位於蒙古國及黑龍江，是金國為防禦塔塔兒部和蒙古諸部在大興安嶺區域所建造的壕塹。
遼代邊壕，又稱成吉思汗邊牆（俄語：Вал_Чингисхана）。
...
```
2. senses are too fine-grained
例如我個人認為sense1冰和sense2食用冰塊相去不遠，可以合併成同一條目
這對evaluation也比較有利（如果答案是s1，model預測s2，那也合理地算對）。
不過這種**合併**除了**人工檢視**，我目前沒有想到一個有效的演算法。

```
冰 (消歧義)
維基百科，自由的百科全書
冰可能是：
* 冰，水的固態形式。
* 食用冰塊，供食用的冰塊。
* 冰毒，一種毒品。
* 冰 (X檔案)，美國科幻電視劇《X檔案》第一季的第八集。
```




In [ ]:
# https://stackoverflow.com/questions/35648648/how-to-get-all-pages-referenced-from-some-wikipedia-page
# MediaWiki API
#  https://zh.wikipedia.org/w/api.php?action=query&prop=links&pllimit=500&plnamespace=0&titles=%E8%98%8B%E6%9E%9C
import requests
from bs4 import BeautifulSoup 
import re
import string
disambig_url = 'https://zh.wikipedia.org/wiki/%E8%8B%B9%E6%9E%9C_(%E6%B6%88%E6%AD%A7%E4%B9%89)'
page = requests.get(disambig_url)
soup = BeautifulSoup(page.content, 'html.parser')
alllines = soup.find_all('li')
for line in alllines:
  # print(line)
  pass
print(soup.prettify())
#長城：<dt>地点</dt> # dot 
#蘋果：soup = BeautifulSoup(x, "lxml")
# for tag in soup.find_all("span") :
#    print(tag.get('id'))
# 彩虹，同蘋果
# debug: 把section加上去（coarse-grained）

In [ ]:
!pip3 -q install Wikipedia-API
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('zh')
apple = wiki_wiki.page("蘋果 (消歧義)")
rd = wiki_wiki.page("彩虹 (消歧義)")
greatwall = wiki_wiki.page("長城 (消歧義)")
def print_sections(sections, level=0):
      for s in sections:
          print("%s: %s" % ("*" * (level + 1), s.title))
          partial_entries = [entry[0:10] for entry in s.text.split()]
          print(partial_entries)
          print('-----------------')
          # print_sections(s.sections, level + 1)
# print_sections(rd.sections) 
print_sections(greatwall.sections) 
print_sections(apple.sections)
# https://www.wikiwand.com/zh-tw/Category:%E5%85%A8%E9%83%A8%E6%B6%88%E6%AD%A7%E7%BE%A9%E9%A0%81%E9%9D%A2

*: 相關條目
['以「长城」開頭的条目', '以「長城」開頭的条目']
-----------------
*: 公司
['蘋果公司，著名电子产', '蘋果園區，苹果公司于', '蘋果唱片公司，披頭士']
-----------------
*: 报纸
['苹果日报', '蘋果日報', '(香港)，香港公司壹', '蘋果日報', '(臺灣)，香港公司壹', '蘋果日報慈善基金，香']
-----------------
*: 电影
['苹果', '(电影)，2007年', '苹果', '(南韩电影)，200']
-----------------
*: 艺人
['拉芘莎拉·瑛特勒素，', '黃暐婷', '，臺灣女性藝人，藝名']
-----------------
*: 参见
['以「苹果」開頭的条目', '名稱包含「苹果」的頁']
-----------------


In [ ]:
app = wiki_wiki.page("蘋果")
app.fullurl[24:]
#len('https://zh.wikipedia.org')

'/wiki/%E8%8B%B9%E6%9E%9C'

In [23]:
# !pip install opencc-python-reimplemented
from opencc import OpenCC
cc = OpenCC('s2t')
def GetCoarseSense(fine_sense, disambig_page):
      '''Only works when the disambiguation page is properly sectioned'''
      coarse_senses = []
      for s in disambig_page.sections:
          coarse_sense = s.title
          # print(fine_sense,"-----", s.title)
          t_s_text = cc.convert(s.text)
          print(coarse_sense, t_s_text)
          if (fine_sense in t_s_text):
            return cc.convert(coarse_sense)
      return ''
fine_sense = '苹果 (南韓电影)'# 苹果 (电影)
GetCoarseSense(fine_sense, apple)

公司 蘋果公司，著名電子產品生產商
蘋果園區，蘋果公司於2017年4月起啓用的公司總部新址
蘋果唱片公司，披頭士樂團創立的唱片公司
报纸 蘋果日報
蘋果日報 (香港)，香港公司壹傳媒在香港發行的報紙
蘋果日報 (臺灣)，香港公司壹傳媒在臺灣發行的報紙
蘋果日報慈善基金，香港一個慈善基金，由壹傳媒有限公司於1995年成立
电影 蘋果 (電影)，2007年上映的中國電影
蘋果 (南韓電影)，2008年上映的南韓電影
艺人 拉芘莎拉·瑛特勒素，泰國女演員、歌手，小名Apple
黃暐婷 ，臺灣女性藝人，藝名apple
参见 以「蘋果」開頭的條目
名稱包含「蘋果」的頁面


''